In [31]:
import os, time, math, numpy as np, torch
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel, GPT2Config

# ------------------------------
# Utils
# ------------------------------

def corr_matrix_fast(A, B):
    A0 = A - A.mean(axis=0, keepdims=True)
    B0 = B - B.mean(axis=0, keepdims=True)
    As = A0.std(axis=0, ddof=1, keepdims=True); As[As==0]=1.0
    Bs = B0.std(axis=0, ddof=1, keepdims=True); Bs[Bs==0]=1.0
    Az = A0/As; Bz = B0/Bs
    return (Az.T @ Bz) / (A.shape[0]-1)

def detect_ffn_layout(block):
    s1 = tuple(block.mlp.c_fc.weight.shape)
    s2 = tuple(block.mlp.c_proj.weight.shape)
    # Two possibilities for GPT2-base:
    # standard: c_fc=[d_ff, d_model], c_proj=[d_model, d_ff]
    # transposed: c_fc=[d_model, d_ff], c_proj=[d_ff, d_model]
    d1, d2 = s1
    if d1 > d2 and s2 == (d2, d1):
        return "standard", d1, d2  # d_ff, d_model
    elif d2 > d1 and s2 == (d2, d1):
        return "transposed", d2, d1 # d_ff, d_model
    else:
        # fall back by comparing which dim matches embed size
        d_model_guess = block.ln_2.weight.shape[0]
        if s1[0] == d_model_guess:
            return "transposed", s1[1], s1[0]
        else:
            return "standard", s1[0], s1[1]

def hook_pre_ffn(block_id, store):
    def _hook(module, inp, out):
        store[block_id].append(out.detach().cpu())
    return _hook

def build_feature_mats(model, tokenizer, texts, max_len=128, max_rows=4000):
    device = next(model.parameters()).device
    model.eval()
    # storage
    store = {i: [] for i in range(len(model.h))}
    hooks = []
    for i, block in enumerate(model.h):
        hooks.append(block.mlp.c_fc.register_forward_hook(hook_pre_ffn(i, store)))
    # tokenize
    enc = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=max_len)
    enc = {k:v.to(device) for k,v in enc.items()}
    with torch.no_grad():
        _ = model(**enc)
    for h in hooks: h.remove()

    feats = {}
    for i in range(len(model.h)):
        X = torch.cat(store[i], dim=0)  # [B, T, d_ff]
        X = X.reshape(-1, X.shape[-1])  # [N, d_ff]
        if X.shape[0] > max_rows:
            X = X[:max_rows]
        feats[i] = X.numpy()
    return feats

def align_layers(Xa, Xb):
    from scipy.optimize import linear_sum_assignment
    C = corr_matrix_fast(Xa, Xb)  # [d,d]
    r, c = linear_sum_assignment(-C)
    return c, float(C[r, c].mean()), C

# Apply permutation to **one** block, handling both layouts.
def apply_ffn_permutation_block(model, layer_idx, perm):
    blk = model.h[layer_idx].mlp
    layout, d_ff, d_model = detect_ffn_layout(model.h[layer_idx])
    p = torch.as_tensor(perm, dtype=torch.long, device=blk.c_fc.weight.device)
    with torch.no_grad():
        if layout == "standard":
            # c_fc: [d_ff, d_model] rows; c_proj: [d_model, d_ff] cols
            blk.c_fc.weight[:]  = blk.c_fc.weight[p, :]
            blk.c_fc.bias[:]    = blk.c_fc.bias[p]
            blk.c_proj.weight[:] = blk.c_proj.weight[:, p]
        else:
            # transposed: c_fc: [d_model, d_ff] cols; c_proj: [d_ff, d_model] rows
            blk.c_fc.weight[:]   = blk.c_fc.weight[:, p]
            blk.c_fc.bias[:]     = blk.c_fc.bias[p]
            blk.c_proj.weight[:] = blk.c_proj.weight[p, :]
    return layout, d_ff, d_model

# Merge FFN of 'src_idx' into 'dst_idx' (weights already aligned by permutation).
# Simple average merge: W1, b1, W2, b2.
def merge_ffn_pair(model, dst_idx, src_idx):
    dst = model.h[dst_idx].mlp
    src = model.h[src_idx].mlp
    with torch.no_grad():
        dst.c_fc.weight[:]   = 0.5 * (dst.c_fc.weight + src.c_fc.weight)
        dst.c_fc.bias[:]     = 0.5 * (dst.c_fc.bias   + src.c_fc.bias)
        dst.c_proj.weight[:] = 0.5 * (dst.c_proj.weight + src.c_proj.weight)
        dst.c_proj.bias[:]   = 0.5 * (dst.c_proj.bias   + src.c_proj.bias)

# Build a compressed GPT-2 (half the layers) and copy weights from anchors.
def build_compressed_gpt2(original, keep_layers):
    device = next(original.parameters()).device
    cfg = original.config
    new_cfg = GPT2Config(
        vocab_size=cfg.vocab_size,
        n_positions=cfg.n_positions,
        n_ctx=cfg.n_ctx,
        n_embd=cfg.n_embd,
        n_layer=len(keep_layers),
        n_head=cfg.n_head,
        n_inner=cfg.n_inner,
        activation_function=cfg.activation_function,
        resid_pdrop=cfg.resid_pdrop,
        embd_pdrop=cfg.embd_pdrop,
        attn_pdrop=cfg.attn_pdrop,
        layer_norm_epsilon=cfg.layer_norm_epsilon,
        initializer_range=cfg.initializer_range,
        bos_token_id=getattr(cfg, "bos_token_id", None),
        eos_token_id=getattr(cfg, "eos_token_id", None),
    )
    tiny = GPT2LMHeadModel(new_cfg).to(device)
    # embeddings / ln_f / lm_head
    with torch.no_grad():
        tiny.transformer.wte.weight[:] = original.transformer.wte.weight
        tiny.transformer.wpe.weight[:] = original.transformer.wpe.weight[:tiny.transformer.wpe.weight.shape[0]]
        tiny.transformer.ln_f.weight[:] = original.transformer.ln_f.weight
        tiny.transformer.ln_f.bias[:]   = original.transformer.ln_f.bias
        tiny.lm_head.weight[:] = original.lm_head.weight

        # copy blocks
        for new_i, old_i in enumerate(keep_layers):
            src = original.transformer.h[old_i]
            dst = tiny.transformer.h[new_i]
            # attn weights/biases
            for name in ["ln_1","attn","ln_2","mlp"]:
                # ln weights/bias
                if name=="ln_1":
                    dst.ln_1.weight[:] = src.ln_1.weight; dst.ln_1.bias[:] = src.ln_1.bias
                elif name=="ln_2":
                    dst.ln_2.weight[:] = src.ln_2.weight; dst.ln_2.bias[:] = src.ln_2.bias
                elif name=="attn":
                    # qkv + proj
                    dst.attn.c_attn.weight[:] = src.attn.c_attn.weight
                    dst.attn.c_attn.bias[:]   = src.attn.c_attn.bias
                    dst.attn.c_proj.weight[:] = src.attn.c_proj.weight
                    dst.attn.c_proj.bias[:]   = src.attn.c_proj.bias
                elif name=="mlp":
                    dst.mlp.c_fc.weight[:]   = src.mlp.c_fc.weight
                    dst.mlp.c_fc.bias[:]     = src.mlp.c_fc.bias
                    dst.mlp.c_proj.weight[:] = src.mlp.c_proj.weight
                    dst.mlp.c_proj.bias[:]   = src.mlp.c_proj.bias
    return tiny

def count_params(model):
    return sum(p.numel() for p in model.parameters())

# ------------------------------
# Pipeline
# ------------------------------

def main():
    torch.set_grad_enabled(False)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    print("Step 0: load tokenizer/model")
    tok = GPT2Tokenizer.from_pretrained("gpt2")
    tok.pad_token = tok.eos_token
    base = GPT2LMHeadModel.from_pretrained("gpt2").to(device).eval()
    base.config.pad_token_id = tok.eos_token_id

    print("Step 1: sample text from WikiText-2")
    ds = load_dataset("wikitext", "wikitext-2-raw-v1", split="test[:1%]")
    texts = [t for t in ds["text"] if len(t.strip())>0][:64]

    print("Step 2: collect FFN pre-activations")
    feats = build_feature_mats(base.transformer, tok, texts, max_len=96, max_rows=4000)
    print("✓ feature matrices built")

    # We'll merge pairs: (0,1), (2,3), ...
    L = len(base.transformer.h)
    pairs = [(i, i+1) for i in range(0, L, 2) if i+1 < L]
    print("Pairs to merge:", pairs)

    from collections import defaultdict
    perms = {}

    print("Step 3: align each src to its dst (anchor = smaller index in pair)")
    for dst, src in pairs:
        Xa = feats[dst]; Xb = feats[src]
        perm, score, _ = align_layers(Xa, Xb)
        perms[(dst,src)] = perm
        print(f"  L{dst}↔L{src}: mean aligned corr={score:.4f}")

    print("Step 4: apply permutations for each src, then merge FFNs into dst")
    for dst, src in pairs:
        # apply permutation to **source** block to match dst ordering
        layout, d_ff, d_model = apply_ffn_permutation_block(base.transformer, src, perms[(dst,src)])
        # merge (average) src into dst FFN
        merge_ffn_pair(base.transformer, dst, src)
        print(f"  merged FFN of L{src} into L{dst}  | layout={layout}, d_ff={d_ff}, d_model={d_model}")

    print("Step 5: build a compressed GPT-2 with half the layers, copying anchors")
    keep_layers = [i for i in range(0, L, 2)]  # keep 0,2,4,...
    tiny = build_compressed_gpt2(base, keep_layers).eval()

    # Save models
    os.makedirs("gpt2_merged_full", exist_ok=True)
    os.makedirs("gpt2_compressed_half", exist_ok=True)
    base.save_pretrained("gpt2_merged_full")
    tok.save_pretrained("gpt2_merged_full")
    tiny.save_pretrained("gpt2_compressed_half")
    tok.save_pretrained("gpt2_compressed_half")

    print("Step 6: parameter counts")
    orig = GPT2LMHeadModel.from_pretrained("gpt2").to(device).eval()
    merged_full = GPT2LMHeadModel.from_pretrained("gpt2_merged_full").to(device).eval()
    compressed = GPT2LMHeadModel.from_pretrained("gpt2_compressed_half").to(device).eval()

    n_orig = count_params(orig)
    n_merged_full = count_params(merged_full)
    n_compressed = count_params(compressed)

    print(f"Params - original:   {n_orig:,}")
    print(f"Params - merged_full (same depth): {n_merged_full:,}  (should match original; merging alone doesn’t cut params)")
    print(f"Params - compressed_half (half depth): {n_compressed:,}  (reduced)")

    # quick sanity generation
    prompt = "The future of artificial intelligence in India"
    inp = tok(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        out_orig = orig.generate(**inp, max_length=40, do_sample=False)
        out_comp = compressed.generate(**inp, max_length=40, do_sample=False)

    print("\nOriginal:\n", tok.decode(out_orig[0], skip_special_tokens=True))
    print("\nCompressed (half layers):\n", tok.decode(out_comp[0], skip_special_tokens=True))

if __name__ == "__main__":
    main()


Step 0: load tokenizer/model
Step 1: sample text from WikiText-2
Step 2: collect FFN pre-activations
✓ feature matrices built
Pairs to merge: [(0, 1), (2, 3), (4, 5), (6, 7), (8, 9), (10, 11)]
Step 3: align each src to its dst (anchor = smaller index in pair)
  L0↔L1: mean aligned corr=0.3734
  L2↔L3: mean aligned corr=0.5047
  L4↔L5: mean aligned corr=0.5289
  L6↔L7: mean aligned corr=0.5190
  L8↔L9: mean aligned corr=0.5035
  L10↔L11: mean aligned corr=0.5042
Step 4: apply permutations for each src, then merge FFNs into dst
  merged FFN of L1 into L0  | layout=transposed, d_ff=3072, d_model=768
  merged FFN of L3 into L2  | layout=transposed, d_ff=3072, d_model=768
  merged FFN of L5 into L4  | layout=transposed, d_ff=3072, d_model=768
  merged FFN of L7 into L6  | layout=transposed, d_ff=3072, d_model=768
  merged FFN of L9 into L8  | layout=transposed, d_ff=3072, d_model=768
  merged FFN of L11 into L10  | layout=transposed, d_ff=3072, d_model=768
Step 5: build a compressed GPT-2 w

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Params - original:   124,439,808
Params - merged_full (same depth): 124,439,808  (should match original; merging alone doesn’t cut params)
Params - compressed_half (half depth): 81,912,576  (reduced)

Original:
 The future of artificial intelligence in India is uncertain. The Indian government has been trying to develop a new AI system that can be used to solve problems in the field of AI. The Indian government has been

Compressed (half layers):
 The future of artificial intelligence in India lasted forever

































In [36]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset
import math

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Load dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
texts = [t for t in dataset['text'] if len(t.strip()) > 0][:100]  # use 100 lines for speed

def compute_perplexity(model_name, texts, batch_size=4):
    model = GPT2LMHeadModel.from_pretrained(model_name).to(device).eval()
    tokenizer.pad_token = tokenizer.eos_token

    total_loss = 0
    total_tokens = 0

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)

        with torch.no_grad():
            loss = model(**inputs, labels=inputs["input_ids"]).loss

        num_tokens = inputs["input_ids"].numel()
        total_loss += loss.item() * num_tokens
        total_tokens += num_tokens

    avg_loss = total_loss / total_tokens
    ppl = math.exp(avg_loss)
    return ppl

orig_ppl = compute_perplexity("gpt2", texts)
compressed_ppl = compute_perplexity("gpt2_compressed_half", texts)

print(f"Original GPT-2 Perplexity: {orig_ppl:.2f}")
print(f"Compressed GPT-2 (k-merged) Perplexity: {compressed_ppl:.2f}")


Original GPT-2 Perplexity: 851.85
Compressed GPT-2 (k-merged) Perplexity: 23989.89


In [ ]:
!pip install transformers datasets accelerate evaluate

In [43]:


import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import math

device = "cuda" if torch.cuda.is_available() else "cpu"

# --------------------------
# 1. Load Models
# --------------------------
orig_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
compressed_model = GPT2LMHeadModel.from_pretrained("gpt2_compressed_half").to(device)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# --------------------------
# 2. Load WikiText-2 Dataset
# --------------------------
# --------------------------
# 2. Load + preprocess WikiText-2 properly
# --------------------------
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

def tokenize_function(examples):
    out = tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True, 
        max_length=128
    )
    out["labels"] = out["input_ids"].copy()
    return out

tokenized = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized.set_format(type="torch")
train_data = tokenized["train"]
test_data = tokenized["test"]



# --------------------------
# 3. Training Function
# --------------------------
def train_model(model, save_dir):
    # Make sure gradients are enabled and model is in training mode
    torch.set_grad_enabled(True)
    model.train()

    args = TrainingArguments(
        output_dir=save_dir,
        eval_strategy="no",
        per_device_train_batch_size=2,
        num_train_epochs=2,
        logging_steps=2000,
        learning_rate=5e-5,
        weight_decay=0.01,
        report_to="none",
        
        # ↓ Add these lines ↓
        save_strategy="steps",    # or "epoch" or "no"
        save_steps=20000,         # save every 50k steps
        save_total_limit=1,       # keep only the last checkpoint
    )


    trainer = Trainer(model=model, args=args, train_dataset=train_data)
    trainer.train()

    model.save_pretrained(save_dir)


# --------------------------
# 4. Train both models
# --------------------------
print("Training original GPT-2...")
train_model(orig_model, "gpt2_finetuned_orig")

print("Training compressed GPT-2...")
train_model(compressed_model, "gpt2_finetuned_compressed")

# --------------------------
# 5. Perplexity Evaluation
# --------------------------
def compute_perplexity(model, data):
    metric = evaluate.load("perplexity")
    model.eval()
    results = metric.compute(model_id=None, model=model, tokenizer=tokenizer, add_start_token=False, texts=data["text"][:200])
    return results["perplexity"]

orig_ppl = compute_perplexity(orig_model, dataset["test"])
compressed_ppl = compute_perplexity(compressed_model, dataset["test"])

print(f"Perplexity (Original GPT-2 after training): {orig_ppl:.2f}")
print(f"Perplexity (Compressed GPT-2 after training): {compressed_ppl:.2f}")


Training original GPT-2...


Step,Training Loss
2000,1.368800
4000,1.333800
6000,1.270700
8000,1.304900
10000,1.274800
12000,1.228300
14000,1.310800
16000,1.246000
18000,1.262800
20000,1.130100


Training compressed GPT-2...


Step,Training Loss
2000,1.977000
4000,1.788100
6000,1.670100
8000,1.691000
10000,1.627900
12000,1.566600
14000,1.656900
16000,1.564000
18000,1.581100
20000,1.465800


ValueError: Evaluation module cache file doesn't exist. Please make sure that you call `add` or `add_batch` at least once before calling `compute`.

In [48]:
import torch
import math
from torch.utils.data import DataLoader
from tqdm import tqdm

def compute_perplexity(model, data, tokenizer, max_samples=200, batch_size=2):
    model.eval()
    model.to(device)
    
    # Take up to `max_samples` non-empty text samples
    texts = [t for t in data["text"] if isinstance(t, str) and len(t.strip()) > 0][:max_samples]
    
    total_loss = 0.0
    total_tokens = 0

    for i in tqdm(range(0, len(texts), batch_size), desc="Computing perplexity"):
        batch = texts[i:i + batch_size]
        encodings = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        
        with torch.no_grad():
            outputs = model(**encodings, labels=encodings["input_ids"])
            loss = outputs.loss
            # Multiply by number of tokens to get total weighted loss
            total_loss += loss.item() * encodings["input_ids"].numel()
            total_tokens += encodings["input_ids"].numel()

    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)
    return perplexity


In [49]:
orig_ppl = compute_perplexity(orig_model, dataset["test"], tokenizer)
compressed_ppl = compute_perplexity(compressed_model, dataset["test"], tokenizer)

print(f"Perplexity (Original GPT-2 after training): {orig_ppl:.2f}")
print(f"Perplexity (Compressed GPT-2 after training): {compressed_ppl:.2f}")


Computing perplexity: 100%|██████████| 100/100 [00:00<00:00, 105.05it/s]

Perplexity (Original GPT-2 after training): 12.28
Perplexity (Compressed GPT-2 after training): 18.93


In [50]:
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

orig_total, orig_trainable = count_parameters(orig_model)
comp_total, comp_trainable = count_parameters(compressed_model)

print(f"Original GPT-2 total params: {orig_total:,}")
print(f"Original GPT-2 trainable params: {orig_trainable:,}")
print(f"Compressed GPT-2 total params: {comp_total:,}")
print(f"Compressed GPT-2 trainable params: {comp_trainable:,}")

reduction = 100 * (1 - comp_total / orig_total)
print(f"Parameter reduction: {reduction:.2f}%")


Original GPT-2 total params: 124,439,808
Original GPT-2 trainable params: 124,439,808
Compressed GPT-2 total params: 81,912,576
Compressed GPT-2 trainable params: 81,912,576
Parameter reduction: 34.17%


In [51]:
import time
import torch

def measure_latency(model, tokenizer, device="cuda", num_samples=20, max_length=64):
    model.eval()
    model.to(device)
    
    # Create random sample inputs (or you can use your dataset)
    texts = ["The quick brown fox jumps over the lazy dog."] * num_samples

    # Warm-up (to avoid first-run overhead)
    for _ in range(3):
        _ = model.generate(
            **tokenizer(texts[0], return_tensors="pt").to(device),
            max_length=max_length
        )

    torch.cuda.synchronize() if device == "cuda" else None

    start_time = time.time()
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt").to(device)
        with torch.no_grad():
            _ = model.generate(**inputs, max_length=max_length)
    torch.cuda.synchronize() if device == "cuda" else None

    total_time = time.time() - start_time
    avg_latency = total_time / num_samples
    return avg_latency

orig_latency = measure_latency(orig_model, tokenizer, device=device)
comp_latency = measure_latency(compressed_model, tokenizer, device=device)

print(f"Average latency (Original GPT-2): {orig_latency*1000:.2f} ms/sample")
print(f"Average latency (Compressed GPT-2): {comp_latency*1000:.2f} ms/sample")

speedup = orig_latency / comp_latency
print(f"Speedup: {speedup:.2f}x faster")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Average latency (Original GPT-2): 14.27 ms/sample
Average latency (Compressed GPT-2): 127.10 ms/sample
Speedup: 0.11x faster


In [7]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
import evaluate
import numpy as np
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load dataset (CNN/DailyMail test split)
data = load_dataset("cnn_dailymail", "3.0.0", split="test[:100]")  # only first 100 for speed

# Load ROUGE metric
rouge = evaluate.load("rouge")

# Load tokenizer
tok = GPT2Tokenizer.from_pretrained("gpt2")
tok.pad_token = tok.eos_token

# Models to test
models = {
    "original_gpt2": "gpt2",
    "compressed_gpt2": "gpt2_compressed_half"
}

def generate_summary(model, text, max_new_tokens=60):
    inp = tok(
        text, return_tensors="pt",
        truncation=True, padding=True, max_length=128
    ).to(device)

    output = model.generate(
        **inp,
        max_new_tokens=max_new_tokens,     # ✅ instead of max_length
        num_beams=3,
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    return tok.decode(output[0], skip_special_tokens=True)

results = {}

for name, path in models.items():
    print(f"\n🔹 Evaluating ROUGE for: {name}")
    model = GPT2LMHeadModel.from_pretrained(path).to(device).eval()

    predictions = []
    references = []

    for i in tqdm(range(len(data))):
        article = data[i]["article"]
        ref = data[i]["highlights"]

        pred = generate_summary(model, article)
        predictions.append(pred)
        references.append(ref)

    score = rouge.compute(predictions=predictions, references=references)
    results[name] = score
    print(f"✅ ROUGE scores for {name}:")
    print(score)

print("\n📊 Final ROUGE Comparison:")
for model_name, score in results.items():
    print(model_name, "→ ROUGE-L:", score["rougeL"])



🔹 Evaluating ROUGE for: original_gpt2


 82%|████████▏ | 82/100 [00:25<00:05,  3.24it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 84%|████████▍ | 84/100 [00:25<00:04,  3.90it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
100%|██████████| 100/100 [00:29<00:00,  3.35it/s]


✅ ROUGE scores for original_gpt2:
{'rouge1': np.float64(0.23686015230383223), 'rouge2': np.float64(0.10447509226414367), 'rougeL': np.float64(0.1703181031849859), 'rougeLsum': np.float64(0.19939702492949057)}

🔹 Evaluating ROUGE for: compressed_gpt2


100%|██████████| 100/100 [00:21<00:00,  4.76it/s]


✅ ROUGE scores for compressed_gpt2:
{'rouge1': np.float64(0.20007001564974664), 'rouge2': np.float64(0.08813575833280402), 'rougeL': np.float64(0.14050997503585588), 'rougeLsum': np.float64(0.16410708368153099)}

📊 Final ROUGE Comparison:
original_gpt2 → ROUGE-L: 0.1703181031849859
compressed_gpt2 → ROUGE-L: 0.14050997503585588


In [9]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------------------
# 1️⃣ Load tokenizer
# -------------------------------
tok = GPT2Tokenizer.from_pretrained("gpt2")
tok.pad_token = tok.eos_token

# -------------------------------
# 2️⃣ Prepare IMDB dataset
# -------------------------------
ds = load_dataset("imdb")
def tok_fn(e):
    return tok(e["text"], truncation=True, padding="max_length", max_length=128)
ds = ds.map(tok_fn, batched=True, remove_columns=["text"])
ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

collator = DataCollatorWithPadding(tokenizer=tok)

# -------------------------------
# 3️⃣ Sentiment classification wrapper
# -------------------------------
class GPT2ForSentiment(nn.Module):
    def __init__(self, gpt_model):
        super().__init__()
        self.gpt2 = gpt_model
        self.classifier = nn.Linear(self.gpt2.config.n_embd, 2)

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.gpt2(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        # ✅ fix: use final non-padding token, not last fixed position
        lengths = attention_mask.sum(dim=1) - 1
        last_hidden = outputs.hidden_states[-1][torch.arange(input_ids.size(0)), lengths, :]
        logits = self.classifier(last_hidden)
        loss = None
        if labels is not None:
            loss = nn.functional.cross_entropy(logits, labels)
        return {"loss": loss, "logits": logits}

# -------------------------------
# 4️⃣ Training setup (same as pruning config)
# -------------------------------
args = TrainingArguments(
    output_dir="./gpt2_imdb_results",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    learning_rate=5e-5,
    logging_steps=50,
    save_strategy="no",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# -------------------------------
# 5️⃣ Train & Evaluate function
# -------------------------------
def train_and_eval(model, name):
    print(f"\n🔹 Training {name} ...")
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds["train"],
        eval_dataset=ds["test"],
        tokenizer=tok,
        data_collator=collator
    )
    trainer.train()

    # ---- Evaluate ----
    model.eval()
    correct, total = 0, 0
    for batch in torch.utils.data.DataLoader(ds["test"], batch_size=8):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            out = model(batch["input_ids"], batch["attention_mask"])
        preds = out["logits"].argmax(dim=-1)
        correct += (preds == batch["label"]).sum().item()
        total += batch["label"].size(0)
    acc = correct / total
    print(f"✅ {name} Test Accuracy: {acc*100:.2f}%")
    return acc

# -------------------------------
# 6️⃣ Evaluate both models
# -------------------------------
# ---- Merged GPT-2 ----
m_merged = GPT2LMHeadModel.from_pretrained("gpt2_merged_full")
model_merged = GPT2ForSentiment(m_merged.to(device))
acc_merged = train_and_eval(model_merged, "Merged GPT-2")

# ---- Compressed GPT-2 ----
m_compressed = GPT2LMHeadModel.from_pretrained("gpt2_compressed_half")
model_compressed = GPT2ForSentiment(m_compressed.to(device))
acc_compressed = train_and_eval(model_compressed, "Comp ressed GPT-2")

# -------------------------------
# 7️⃣ Summary
# -------------------------------
print("\n📊 Final IMDb Accuracies:")
print(f"  Merged GPT-2:     {acc_merged*100:.2f}%")
print(f"  Compressed GPT-2: {acc_compressed*100:.2f}%")



🔹 Training Merged GPT-2 ...


/tmp/ipykernel_7424/4199903884.py:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,1.471400
100,0.863700
150,0.876700
200,0.745700
250,0.713200
300,0.714900
350,0.788200
400,0.897400
450,0.771100
500,0.771200


✅ Merged GPT-2 Test Accuracy: 83.67%

🔹 Training Compressed GPT-2 ...


Step,Training Loss
50,2.354500
100,0.792000
150,0.773100
200,0.689500
250,0.746300
300,0.697500
350,0.768500
400,0.709200
450,0.627300
500,0.951000


✅ Compressed GPT-2 Test Accuracy: 83.58%

📊 Final IMDb Accuracies:
  Merged GPT-2:     83.67%
  Compressed GPT-2: 83.58%
